# LangChain と pinecone を使った Amazon Bedrock による拡張質問と回答の取得

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

### コンテキスト
以前、モデルがタイヤの交換方法を教えてくれるのを見ましたが、関連データを手動で提供し、コンテキストを自分たちで提供する必要がありました。 Bedrockで利用できるモデルを活用し、トレーニング中に学んだ知識に基づいて質問したり、マニュアルコンテキストを提供したりするアプローチを検討しました。このアプローチは短い文書やシングルトンアプリケーションでは有効ですが、モデルに送られるプロンプトにすべて収まらない大規模な企業文書が存在する場合に、企業レベルの質問への回答には対応できません。 

### パターン
私たちは上記プロセスを Retreival Augmented Generation (RAG)と呼ばれるアーキテクチャを実装することで改善することができます。RAG は言語モデルの外部からデータ（ノンパラメトリック）を取得し、関連する取得データをコンテキストに追加することでプロンプトを補強します。

このノートブックでは、ユーザーの質問に対する回答を提供するためにドキュメントを検索し、活用する質問応答のパターンへのアプローチ方法を説明します。

### 課題
- トークン制限を超える大きな文書をどのように管理するか
- 質問に関連する文書を見つける方法

### 提案
上記の課題に対して、このノートでは以下の戦略を提案します。
#### ドキュメントを準備する
![埋め込み(Embeddings)](./images/Embeddings_lang.png)

質問に答えるには、文書を処理してドキュメントストアインデックスに保存する必要があります。
- 文書をロード
- 処理して小さなチャンクに分割
- Amazon Bedrock Titan Embeddings モデルを使用して、各チャンクの数値ベクトル表現を作成
- チャンクとそれに対応する埋め込みを使用してインデックスを作成
#### 質問する
![質問](./images/Chatbot_lang.png)

ドキュメントインデックスが作成されたら、質問する準備が整い、質問に基づいて関連ドキュメントが取得されます。 以下のステップが実行されます。
- 入力された質問の埋め込みを作成
- 質問の埋め込みと索引の埋め込みを比較
- (上位 N 件) の関連文書チャンクを取得
- それらのチャンクをプロンプトのコンテキストの一部として追加
- Amazon Bedrock 下のモデルにプロンプトを送信
- 取得したドキュメントに基づいて状況に応じた回答を得る

## ユースケース
#### データセット
このアーキテクチャパターンを説明するために、IRS の文書を使用しています。これらの文書には次のようなトピックが説明されています:
- オリジナル・イシュー・ディスカウント (OID) 商品
- 1万ドルを超える現金支払いのIRSへの報告
- 雇用者向け税務ガイド

#### ペルソナ
IRS の仕組みや、何らかの行動が意味を持つかどうかを理解していない素人のペルソナを想定してみましょう。

モデルは簡単な言葉で文書から答えようとします。

## 実装
RAGアプローチを採用するために、このノートブックは LangChain フレームワークを使用しています。LangChain フレームワークでは、RAGなどのパターンを効率的に構築できるさまざまなサービスやツールと統合されています。以下のツールを使用します。

- **LLM (Large Language Model)**: Amazon Bedrock から入手可能な Anthropic Claude V1

  このモデルを使って文書の塊を理解し、わかりやすい方法で答えを出します。
- **Embeddings Model**: Amazon Bedrock から入手可能な Amazon Titan Embeddings

  このモデルを使用して、テキスト文書の数値表現を生成します。
- **Document Loader**: LangChain から入手可能な PDF Loader

  これはソースからドキュメントをロードできるローダーです。このノートブックでは、ローカルパスからサンプルファイルをロードしています。これを、企業の内部システムから文書を読み込むためのローダーに簡単に置き換えることができます。

- **Vector Store**: LangChain から入手可能な FAISS 

  このノートブックでは、このメモリ内ベクトルストアを使用して、埋め込み (Embeddings) とドキュメントの両方を保存しています。エンタープライズ環境では、これを AWS OpenSearch、PgVector 搭載の RDS Postgres、ChromaDB、Pinecone、Weaviate などの永続的なストアに置き換えることができます。
- **Index**: ベクトルインデックス

  The index helps to compare the input embedding and the document embeddings to find relevant document
- **Wrapper**: Index、Vector Store、Embeddings Model、LLMをラップして、ロジックをユーザーから抽象化します。

この [ノートブック](https://www.pinecone.io/learn/series/langchain/langchain-retrieval-augmentation/) とこの [ノートブック](01_qa_w_rag_claude.ipynb) のアイデアの助けを借りて構築されました

## セットアップ

このノートブックの残りの部分を実行する前に、以下のセルを実行して (必要なライブラリがインストールされていることを確認し) Bedrockに接続する必要があります。

For more details on how the setup works and ⚠️ **whether you might need to make any changes**, refer to the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) notebook.
セットアップの仕組みと ⚠️ **変更が必要かどうかについての詳細**は、[Bedrock boto3 セットアップノートブック](../00_Intro/Bedrock_boto3_Setup.ipynb) を参照してください。

このノートブックでは、いくつかの追加の依存関係も必要になります。

- [Pinecone](http://pinecone.io), ベクター埋め込みを保存
- [PyPDF](https://pypi.org/project/pypdf/), PDF ファイルの処理用

### Pinecone 前提条件
- app.pinecone.io の Pinecone API キーを環境変数 PINECONE_API_KEY として設定
- Pinecone 環境 - コンソールで API キーの横を検索し、そのキーを環境変数 PINECONE_ENVIRONMENT として設定します

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

In [ ]:
%pip install -U "faiss-cpu>=1.7,<2" langchain==0.0.309 "pypdf>=3.8,<4" \
    pinecone-client \
    apache-beam \
    datasets \
    tiktoken

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ 必要に応じて AWS 設定に関する以下のコードのコメントを解除、編集してください ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## Langchainの設定

まず、LLM と埋め込みモデルをインスタンス化します。ここでは、テキスト生成には Anthropic Claude を使用し、テキストの埋め込みには Amazon Titan を使用しています。

注:Bedrock では他のモデルも選択できます。`model_id`を以下のように置き換えてモデルを変更することができます。

`llm = Bedrock(model_id="amazon.titan-tg1-large")`

利用可能な `model_id`:

- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v1`
- `anthropic.claude-v1`

In [ ]:
# Titan Embeddings モデルを使用して埋め込みを生成します。
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# - Anthropic モデルの作成
llm = Bedrock(
    model_id="anthropic.claude-v1", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 200}
)
bedrock_embeddings = BedrockEmbeddings(client=boto3_bedrock)

## データの準備
まず、いくつかのファイルをダウンロードして、ドキュメントストアを構築しましょう。この例では、[ここ](https://www.irs.gov/publications) にある公開の IRS 文書を使用します。

In [ ]:
from urllib.request import urlretrieve

os.makedirs("data", exist_ok=True)
files = [
    "https://www.irs.gov/pub/irs-pdf/p1544.pdf",
    "https://www.irs.gov/pub/irs-pdf/p15.pdf",
    "https://www.irs.gov/pub/irs-pdf/p1212.pdf",
]
for url in files:
    file_path = os.path.join("data", url.rpartition("/")[2])
    urlretrieve(url, file_path)

ダウンロード後、[LangChainで利用可能なPyPDFのディレクトリローダー](https://python.langchain.com/en/latest/reference/modules/document_loaders.html)の助けを借りてドキュメントをロードし、小さなチャンクに分割することができます。

注：検索された文書/テキストは、質問に答えるのに十分な情報を含む大きさでなければなりませんが、LLMプロンプトに収まる大きさであれば十分です。また、埋め込みモデルでは、入力トークンの長さを512トークンまでに制限しています。このユースケースのために、私たちは[RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)を使用して、100文字のオーバーラップで約1000文字のチャンクを作成しています。

In [ ]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data/")

documents = loader.load()
# - 私たちのテストでは、文字分割はこのPDFデータセットの方がうまく機能します
text_splitter = RecursiveCharacterTextSplitter(
    # 見せるために、本当に小さいチャンクサイズを設定してください。
    chunk_size=1000,
    chunk_overlap=100,
)
docs = text_splitter.split_documents(documents)

In [ ]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents]) // len(
    documents
)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f"Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.")
print(f"After the split we have {len(docs)} documents more than the original {len(documents)}.")
print(
    f"Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters."
)

In [ ]:
try:
    
    sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
    modelId = bedrock_embeddings.model_id
    print("Embedding model Id :", modelId)
    print("Sample embedding of a document chunk: ", sample_embedding)
    print("Size of the embedding: ", sample_embedding.shape)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

同様のパターン埋め込みをコーパス全体に対して生成し、ベクトルストアに保存することができます。

これは、[LangChain](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/faiss.html)内の[Pinecone](https://python.langchain.com/docs/integrations/vectorstores/pinecone)実装を使うことで、簡単に実現できます。この実装は、埋め込みモデルとドキュメントを入力として受け取り、ベクトルストア全体を作成します。Index Wrapperを使うことで、プロンプトの作成、クエリの埋め込みデータの取得、関連するドキュメントのサンプリング、LLMの呼び出しといった重い作業のほとんどを抽象化することができます。[VectorStoreIndexWrapper](https://python.langchain.com/en/latest/modules/indexes/getting_started.html#one-line-index-creation)がその手助けをしてくれます。

In [ ]:
import pinecone
import time
import os

# app.pinecone.io からPinecone API キーを追加
api_key = os.environ.get("PINECONE_API_KEY") or "YOUR_API_KEY"
# Pinecone 環境の設定-コンソールの API キーの横を検索
env = os.environ.get("PINECONE_ENVIRONMENT") or "YOUR_ENV"

pinecone.init(api_key=api_key, environment=env)


if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

pinecone.create_index(name=index_name, dimension=sample_embedding.shape[0], metric="dotproduct")
# インデックスの初期化が完了するまで待つ
while not pinecone.describe_index(index_name).status["ready"]:
    time.sleep(1)

In [ ]:
index = pinecone.Index(index_name)
index.describe_index_stats()

**⚠️⚠️⚠️ 注:次のセルの実行には数分かかる場合があります ⚠️⚠️⚠️**

In [ ]:
%%time

from tqdm.auto import tqdm
from uuid import uuid4
from langchain.vectorstores import Pinecone

batch_limit = 50

texts = []
metadatas = []

for i, record in enumerate(tqdm(data)):
    # 最初にこのレコードのメタデータフィールドを取得
    metadata = {"wiki-id": str(record["id"]), "source": record["url"], "title": record["title"]}
    # 今度はレコードテキストからチャンクを作成します
    record_texts = text_splitter.split_text(record["text"])
    # チャンクごとに個別のメタデータ辞書を作成
    record_metadatas = [
        {"chunk": j, "text": text, **metadata} for j, text in enumerate(record_texts)
    ]
    # これらを現在のバッチに追加
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)
    # batch_limit に達したら、テキストを追加できます
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        # embeds = embed.embed_documents(texts)
        embeds = np.array([np.array(bedrock_embeddings.embed_query(text)) for text in texts])
        index.upsert(vectors=zip(ids, embeds.tolist(), metadatas))
        texts = []
        metadatas = []

In [ ]:
index.describe_index_stats()

## LangChain ベクトルストアとクエリ

インデックスは LangChain とは独立して構築されます。これは単純なプロセスであり、Pinecone クライアントで直接行うほうが速いからです。ただし、LangChain に戻ろうとしているので、LangChain ライブラリーを介してインデックスに再接続する必要があります。

In [ ]:
from langchain.vectorstores import Pinecone

text_field = "text"

# langchainの通常のインデックスに戻す
index = pinecone.Index(index_name)

vectorstore = Pinecone(index, bedrock_embeddings.embed_query, text_field)

#### 類似性検索メソッドを使用すると、LLM がレスポンスを生成しなくても、直接クエリを実行してテキストのチャンクを返すことができます。

In [ ]:
query = "Is it possible that I get sentenced to jail due to failure in filings?"

vectorstore.similarity_search(query, k=3)  # 検索クエリ # 最も関連性の高い 3 つのドキュメントを返す

#### これらはすべて関連する結果であり、システムの検索コンポーネントが機能していることを示しています。次のステップは、取得したコンテキストで提供された情報を使用して質問に生成的に回答する LLM を追加することです。

## 生成型質問応答 (Generative Question Answering)

生成型質問応答 (GQA) では、質問をClaude-2に渡しますが、ナレッジベースから返された情報に基づいて回答するように指示します。LangChain では RetrievalQA チェーンを使ってこれを簡単に行うことができます。

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

#### これを先ほどのクエリで試してみましょう:

In [ ]:
qa.run(query)

### 今回のレスポンスは、ベクトルデータベースから取得した情報に基づいてgpt-3.5-turbo LLMが生成したものです。

私たちはまだ、モデルによる説得力のある間違ったハルシネーションから完全に守られているわけではありません。しかし、私たちは提供される答えに対する信頼を高めるためにもっとできることがあります。

そのための効果的な方法は、回答に引用を加えることで、ユーザーがその情報の出所を確認できるようにすることです。これは、RetrievalQAWithSourcesChainと呼ばれるRetrievalQAチェーンの少し異なるバージョンを使用して行うことができます。

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever()
)

In [ ]:
qa_with_sources(query)

#### これで、質問への回答ができました。また、LLMが使用しているこの情報の出所も記載しました。

#### ベクトルデータベースをナレッジベースとして使用して、ソース知識に基づいて大規模言語モデルをベースにする方法を学びました。これを利用することで、すべての回答に引用を提供することで、LLMの回答の正確性を高め、情報源の知識を最新の状態に保ち、システムへの信頼性を高めることができます。

このクエリの埋め込みを使用して、関連するドキュメントを取得できます。
クエリが埋め込みとして表示されるようになったので、クエリをデータストアと類似検索して、最も関連性の高い情報を得ることができます。

### カスタマイズ可能なオプション
上記のシナリオでは、質問に対する文脈を考慮した回答を得るための、迅速で簡単な方法を探りました。次に、[RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html)の助けを借りて、よりカスタマイズ可能なオプションを見てみましょう。ここでは、`chain_type`パラメータを使用して、取得したドキュメントをプロンプトに追加する方法をカスタマイズすることができます。また、何件の関連文書を取得するかを制御したい場合は、以下のセルの `k` パラメータを変更すると、異なる出力を見ることができます。多くのシナリオでは、LLM が答えを生成するために使用したソース文書がどれであるかを知りたい場合があります。また、`RetrievalQA`では、モデルに固有のカスタム[プロンプトテンプレート](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html) を指定することができます。

注: この例では、Amazon BedrockのLLMとしてAnthropic Claudeを使用しています。この特定のモデルは、入力が`Human:`の下で提供され、モデルが`Assistant:`の後に出力を生成するように要求された場合に、最高のパフォーマンスを発揮します。下のセルでは、LLMがコンテキストから外れた回答をしないように、プロンプトをコントロールする方法の例を示しています。

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
)
query = "Is it possible that I get sentenced to jail due to failure in filings?"
result = qa({"query": query})
print_ww(result["result"])

In [ ]:
result["source_documents"]

## 結論
検索拡張生成 (RAG) に関するこのモジュールの完了、おめでとうございます！これは、大規模言語モデルのパワーと検索手法の精度を組み合わせた重要なテクニックです。関連する検索例で生成を補強することで、得られた回答はより首尾一貫し、一貫性があり、根拠があるものになります。この革新的なアプローチを学んだことを誇りに思うべきです。あなたが得た知識は、創造的で魅力的な言語生成システムを構築するのに非常に役立つと確信しています。よく頑張りました！

上記のRAGベースの質問応答の実装では、Amazon Bedrock と LangChain の統合を使って、以下のコンセプトとその実装方法を探りました。

- ベクトルストアを作成するために、ドキュメントをロードし、埋め込みを生成する
- 質問に対するドキュメントの取得
- LLMへの入力となるプロンプトを準備する
- 人間にとって使いやすい方法で答えを提示する
- すべての回答に引用を提供することで、ソース知識を最新の状態に保ち、システムの信頼性を向上させる

### 課題
- 様々なベクトルストアを試す
- Amazon Bedrockで利用可能な様々なモデルを活用し、別の出力を確認する
- 埋め込みやドキュメントチャンクの永続的な保存などのオプションを検討する
- エンタープライズデータストアとの統合

# ありがとうございました。